# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import lib
importlib.reload(lib)

<module 'lib' from 'c:\\Users\\dongq\\OneDrive\\Desktop\\New results\\Portfolio Allocation\\Learning-the-Optimal-Solution-Path\\lib\\__init__.py'>

In [6]:
from lib.lsp.basis_generator import bivariate_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import adaptive_lsp
from lib.lsp.loss_fn_lsp import allocation_cost
from lib.lsp.utils_lsp import get_sup_error_lsp_2d
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Load data

In [7]:
# file path for Colab. May need to change this
# X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
# y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')
decomp_cov_df = pd.read_csv('decomp_pert_10_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [8]:
decomp_cov = np.array(decomp_cov_df)
mean = np.array(mean_df).squeeze()

In [9]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [10]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

In [11]:
lam_max_2d = [1, 1]
lam_min_2d = [.2, 0]
input_dim = decomp_cov.shape[1] - 1

In [12]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list_pert_10_1_20_20.csv')

# Display the DataFrame
truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,13.732664,13.776362,13.820001,13.863581,13.907102,13.950563,13.993965,14.037308,14.080591,14.123816,14.166981,14.210087,14.253134,14.296122,14.339050,14.381919,14.424729,14.467480,14.510172,14.552804
1,13.124255,13.168004,13.211692,13.255319,13.298883,13.342386,13.385827,13.429206,13.472523,13.515779,13.558973,13.602105,13.645175,13.688184,13.731131,13.774016,13.816840,13.859601,13.902301,13.944939
2,12.515792,12.559598,12.603340,12.647017,12.690629,12.734177,12.777660,12.821079,12.864433,12.907723,12.950948,12.994109,13.037205,13.080236,13.123203,13.166106,13.208944,13.251718,13.294427,13.337071
3,11.907270,11.951137,11.994937,12.038670,12.082334,12.125931,12.169461,12.212923,12.256318,12.299645,12.342904,12.386096,12.429220,12.472277,12.515266,12.558188,12.601042,12.643828,12.686547,12.729198
4,11.298679,11.342614,11.386478,11.430271,11.473993,11.517645,11.561225,11.604735,11.648174,11.691541,11.734838,11.778064,11.821220,11.864304,11.907317,11.950260,11.993131,12.035932,12.078662,12.121321
5,10.690007,10.734017,10.777952,10.821812,10.865598,10.909310,10.952946,10.996508,11.039996,11.083409,11.126748,11.170011,11.213201,11.256315,11.299356,11.342321,11.385212,11.428029,11.470771,11.513438
6,10.081243,10.125336,10.169349,10.213284,10.257141,10.300919,10.344618,10.388238,10.431780,10.475243,10.518627,10.561933,10.605160,10.648309,10.691379,10.734370,10.777283,10.820117,10.862872,10.905548
7,9.472370,9.516555,9.560656,9.604675,9.648610,9.692462,9.736230,9.779916,9.823518,9.867037,9.910473,9.953825,9.997095,10.040281,10.083384,10.126404,10.169341,10.212194,10.254964,10.297651
8,8.863368,8.907656,8.951856,8.995968,9.039992,9.083927,9.127774,9.171532,9.215202,9.258784,9.302278,9.345683,9.389000,9.432228,9.475368,9.518420,9.561384,9.604259,9.647046,9.689745
9,8.254211,8.298617,8.342928,8.387146,8.431269,8.475299,8.519234,8.563075,8.606822,8.650475,8.694033,8.737498,8.780869,10.832329,8.867328,8.910416,8.953410,8.996310,9.039116,9.081828


In [13]:
true_losses = truth.to_numpy()

In [14]:
loss_fn = allocation_cost

# 03 Adaptively learn solution path

In [15]:
max_epochs = 100

In [16]:
phi_lam = bivariate_legendre

In [22]:
start_basis_dim = 1
end_basis_dim = 10
init_lr = .5**9

In [18]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 2.5)

In [23]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = adaptive_lsp(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                        gamma=0.9, q=1.2, k_0=5,
                                        intercept=False, thresh_basis=thresh_basis, record_frequency=10,
                                        record_fctn=get_sup_error_lsp_2d,
                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 1 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 0.8744410206403899
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.2993744541731047
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.13764082685376877
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.08112036481763596
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.05785357252027268
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.05584894910718674
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.0518902470197844
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.047921054514901584
--------approximate solution path for # itr = 90 complete

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_exact_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_exact_boosted_legendre.csv'

LSP_results_exact = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,3.808291e+00
1,20.0,4.882355e-01
2,30.0,3.149908e-01
3,40.0,6.118584e-02
4,50.0,4.096654e-02
5,60.0,2.363187e-02
6,70.0,1.431553e-02
7,80.0,9.075150e-03
8,90.0,6.006986e-03
9,100.0,4.100710e-03
